In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")
import os
print(os.getcwd())
import shutil
import tempfile
import time
import matplotlib.pyplot as plt
import numpy as np
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric

from torch.utils.data import DataLoader, Dataset

import torch
import torchvision
import skimage
import pandas as pd
import sklearn
import sklearn.model_selection

from unet.augmentations.augmentations import (
    Compose,
    ToTensor,
    LabelsToEdgesAndCentroids,
    RandomContrastBrightness,
    RandomGuassianBlur,
    RandomGaussianNoise,
    RandomRotate2D,
    Flip,
    RandomRot90,
    RandomPoissonNoise,
    ElasticDeform,
    RandomScale,
    Normalize
)
from unet.utils.load_data import MaddoxDataset
# from threedee.networks.threedee_unet import UNet3D

/home/ubuntu/UNet_3D/threedee_nn


In [4]:
data_transforms = {
    "train": 
        Compose(
            [
                RandomContrastBrightness(p=0.5),
                Flip(p=0.5),
                # RandomScale(scale_limit=[0.5, 1.0], p=0.5),
                RandomRot90(p=0.5),
                RandomGuassianBlur(p=0.5),
                RandomGaussianNoise(p=0.5),
                RandomPoissonNoise(p=0.5),
                ElasticDeform(sigma=5, points=1, p=0.5),
                LabelsToEdgesAndCentroids(centroid_pad=2),
                Normalize(),
                ToTensor()
            ]
        ),
    "val": 
        Compose(
            [
                Normalize(),
                ToTensor()
            ]
        )
}

load_csv = pd.read_csv("patch_data/load_data_training.csv")

train_dataset, val_dataset = sklearn.model_selection.train_test_split(load_csv, test_size=0.2)

train_ds = MaddoxDataset(
    data_csv=train_dataset,
    transform=data_transforms["train"]
    )

val_ds = MaddoxDataset(
    data_csv=val_dataset,
    transform=data_transforms["val"] 
    )

def collate_fn(data):
    """Stack images and masks separately into batches
    (batch, classes, D, H, W)"""
    images = []
    masks = []
    for batch in data:
        image = batch["image"]
        mask = batch["mask"]
        images.append(image)
        masks.append(mask)

    images = torch.stack(images, axis=0)
    masks = torch.stack(masks, axis=0)
    return images, masks

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Don't shuffle validation so you can see how predictions improve over time
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)

data = next(iter(train_loader))

# print(data["image"].shape, data["mask"].shape)

# img = data["image"].numpy()
# mask = data["mask"].numpy()

# print(img.shape, mask.shape)
# fig, ax = plt.subplots(1, 2, figsize=(15, 30))
# ax[0].imshow(img[0,10,...])
# ax[1].imshow(mask[0,10,...])

In [1]:
import sys
sys.path.append("../")
import pandas as pd
from unet.utils.load_data import MaddoxDataset
import sklearn.model_selection
from torch.utils.data import DataLoader

load_csv = pd.read_csv("../patch_data/load_data_training.csv")

train_dataset, val_dataset = sklearn.model_selection.train_test_split(load_csv, test_size=0.2)

train_ds = MaddoxDataset(
    # data_csv=train_dataset,
    csv_path="../patch_data/load_data_training.csv",
    train_val="test"
    )

val_ds = MaddoxDataset(
    # data_csv=val_dataset,
    csv_path="../patch_data/load_data_training.csv",
    # transform=data_transforms["val"] 
    train_val="val"
    )

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=train_ds.collate_fn)

# Don't shuffle validation so you can see how predictions improve over time
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=val_ds.collate_fn)


In [2]:
data = next(iter(train_loader))

print(data["image"].shape, data["mask"].shape)

img = data["image"].numpy()
mask = data["mask"].numpy()

print(img.shape, mask.shape)
fig, ax = plt.subplots(1, 2, figsize=(15, 30))
ax[0].imshow(img[0,10,...])
ax[1].imshow(mask[0,10,...])

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/UNet_3D/threedee_nn/notebooks/patch_data/image/mx85-nd-acqusition-0-ch1_patch70.tif'

In [4]:
# Use MPS backend for M1 Mac
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
        device = torch.device("cpu")
else:
    # device = torch.device("mps")
    device = torch.device("cpu")

print(device)

cpu


In [9]:
model = UNet3D(
    patch_size = (5, 128, 128),
    in_channels=1, 
    out_channels=2
    ).to(device)

loss_function = DiceLoss(smooth_nr=0, smooth_dr=1e-5, squared_pred=True, to_onehot_y=False, sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5)
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)



In [10]:
max_epochs = 600
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []

for epoch in range(max_epochs):
    print(epoch)
    epoch_loss = 0
    for i, data in enumerate(train_ds):
        X, y = data["image"].to(device), data["mask"].to(device)
        model.train()
        prediction = model(X)
        loss = loss_function(prediction, y)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= 0
    epoch_loss_values.append(epoch_loss)

In [5]:
%load_ext autoreload
%autoreload 2
from threedee.networks.threedee_unet import UNet_3D

def train_step(
    model,
    device,
    data_loader,
    loss_fn,
    optimizer
):
    model.train()
    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        prediction = model(X)
        print(prediction.shape, y.shape)
        loss = loss_fn(prediction, y)
        loss.backward()
        optimizer.step()

        probability = prediction.sigmoid().detach().cpu().numpy().flatten()
        y = y.long().detach().cpu().numpy().flatten()

        # roc = sklearn.metrics.roc_auc_score(y, probability, labels=[0, 1], multi_class="ovo")

        th_prediction = (probability > 0.5)
        precision = sklearn.metrics.precision_score(y, th_prediction, labels=[0, 1])
        recall = sklearn.metrics.recall_score(y, th_prediction, labels=[0, 1])
        accuracy = sklearn.metrics.accuracy_score(y, th_prediction)
        f1 = sklearn.metrics.f1_score(y, th_prediction, labels=[0, 1])

        print(
            f"""
            Training step: {i} of {len(data)}
            Loss: {loss}
            Accuracy: {accuracy}
            Precision: {precision}
            Recall: {recall}
            F1: {f1}
            """
        )

def validation_step(
    model,
    device,
    data_loader,
    loss_fn
):
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            X, y = X.to(device), y.to(device)
            prediction = model(X)
            print(prediction.shape, y.shape)
            loss = loss_fn(prediction, y)

            probability = prediction.sigmoid().detach().cpu().numpy().flatten()
            y = y.long().cpu().detach().numpy().flatten()

            # roc = sklearn.metrics.roc_auc_score(y, probability, labels=[0, 1], multi_class="ovo")
            
            th_prediction = (probability > 0.5)
            precision = sklearn.metrics.precision_score(y, th_prediction, labels=[0, 1])
            recall = sklearn.metrics.recall_score(y, th_prediction, labels=[0, 1])
            accuracy = sklearn.metrics.accuracy_score(y, th_prediction)
            f1 = sklearn.metrics.f1_score(y, th_prediction, labels=[0, 1])


            print(
                f"""
                Validation step: {i} of {len(data)}
                Loss: {loss}
                Accuracy: {accuracy}
                Precision: {precision}
                Recall: {recall}
                F1: {f1}
                """
            )

max_epochs = 2

device = "cuda"
model = UNet_3D(
    in_channels=1, 
    out_channels=2
    ).to(device)

loss_function = DiceLoss(smooth_nr=0, smooth_dr=1e-5, squared_pred=True, to_onehot_y=False, sigmoid=False)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5)

for epoch in range(max_epochs):
    print("epoch:", epoch)
    train_step(model, device, train_loader, loss_function, optimizer)
    validation_step(model, device, val_loader, loss_function)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
epoch: 0


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.32 GiB (GPU 0; 14.62 GiB total capacity; 7.97 GiB already allocated; 5.76 GiB free; 8.66 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF